<a href="https://colab.research.google.com/github/fatsasabila/deeplearning_fatsa/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Install required libraries
!pip install transformers

from transformers import pipeline

# Load the sentiment analysis model from Hugging Face
sentiment_model = pipeline("sentiment-analysis")

# Run the model on a sentence
sentence = "I am feeling great today!"
prediction = sentiment_model(sentence)
print(f"Sentence: {sentence}\nPrediction: {prediction}")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Sentence: I am feeling great today!
Prediction: [{'label': 'POSITIVE', 'score': 0.9998760223388672}]


In [24]:
import random

# Create a synthetic dataset
synthetic_data = [
    ("I love this product!", "POSITIVE"),
    ("This is the worst experience ever.", "NEGATIVE"),
    ("I am not sure how I feel about this.", "NEUTRAL"),
    ("The service was excellent!", "POSITIVE"),
    ("I am disappointed with the quality.", "NEGATIVE")
]

# Check the model’s accuracy on the synthetic dataset
correct = 0
for sentence, label in synthetic_data:
    prediction = sentiment_model(sentence)
    predicted_label = prediction[0]['label']
    if predicted_label == label:
        correct += 1

accuracy = correct / len(synthetic_data)
print(f"Accuracy on synthetic dataset: {accuracy * 100:.2f}%")


Accuracy on synthetic dataset: 80.00%


In [26]:
# Install the required datasets library
!pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00


In [28]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_metric

# Load tokenizer and model for sequence classification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenize the synthetic dataset
def tokenize_data(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Convert synthetic data to dataset format
from datasets import Dataset
synthetic_dataset = Dataset.from_dict({
    'text': [s[0] for s in synthetic_data],
    'label': [0 if s[1] == "NEGATIVE" else 1 for s in synthetic_data]
})

# Tokenize the dataset
synthetic_dataset = synthetic_dataset.map(tokenize_data, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch"
)

# Load metric for accuracy evaluation
accuracy_metric = load_metric("accuracy")

# Define a function for computing accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=synthetic_dataset,
    eval_dataset=synthetic_dataset,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
trainer.train()
trainer.evaluate()


ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.10/dist-packages/datasets/__init__.py)

In [30]:
# Install required libraries
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

# Load tokenizer and model for sequence classification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Create synthetic data
synthetic_data = [
    ("I love this product!", 1),  # Positive
    ("This is the worst experience ever.", 0),  # Negative
    ("I am not sure how I feel about this.", 0),  # Neutral -> Negative for simplicity
    ("The service was excellent!", 1),  # Positive
    ("I am disappointed with the quality.", 0)   # Negative
]

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    [s[0] for s in synthetic_data],
    [s[1] for s in synthetic_data],
    test_size=0.2
)

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=64)

# Create PyTorch dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

# Define accuracy calculation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {'accuracy': accuracy}

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.173537,0.000000
2,No log,0.948281,0.000000
3,No log,0.908096,0.000000


Evaluation Results: {'eval_loss': 0.908096194267273, 'eval_accuracy': 0.0, 'eval_runtime': 0.0876, 'eval_samples_per_second': 11.421, 'eval_steps_per_second': 11.421, 'epoch': 3.0}


In [31]:
# Install required libraries
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

# Load tokenizer and model for sequence classification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Create synthetic data
synthetic_data = [
    ("I love this product!", 1),  # Positive
    ("This is the worst experience ever.", 0),  # Negative
    ("I am not sure how I feel about this.", 0),  # Neutral -> Negative for simplicity
    ("The service was excellent!", 1),  # Positive
    ("I am disappointed with the quality.", 0)   # Negative
]

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    [s[0] for s in synthetic_data],
    [s[1] for s in synthetic_data],
    test_size=0.2,
    random_state=42  # Fix the random state for reproducibility
)

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=64)

# Create PyTorch dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,  # Increase number of epochs
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,  # Common learning rate for BERT
)

# Define accuracy calculation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {'accuracy': accuracy}

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.715300,0.000000
2,No log,0.659916,1.000000
3,No log,0.647266,1.000000
4,No log,0.641586,1.000000
5,No log,0.629210,1.000000
6,No log,0.609626,1.000000
7,No log,0.584888,1.000000
8,No log,0.573586,1.000000
9,No log,0.572570,1.000000
10,No log,0.572136,1.000000


Evaluation Results: {'eval_loss': 0.5721359848976135, 'eval_accuracy': 1.0, 'eval_runtime': 0.1368, 'eval_samples_per_second': 7.309, 'eval_steps_per_second': 7.309, 'epoch': 10.0}
